In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sns
%matplotlib inline

from pandas.io.json import json_normalize
import requests # 크롤링을 위함
import json # json파일을 다루기위함
import time # 크롤링에 딜레이를 두기 위함
from tqdm import trange #  크롤링 진행시 progress Bar 

### 크롤링 사이트 주소

https://www.seoul.go.kr/coronaV/coronaStatus.do 발생동향|서울시 코로나 19 

### 크롤링 함수 생성 

In [2]:
# 발생동향|서울시 코로나 19 사이트에서 확진자 상세현황에 있는 데이터를 크콜링 하기 위해서는 두개의 함수 생성이 필요
# URL 정의가 다르기 때문에 각각에 맞는 URL를 넣어서 함수 생성 

# 과거120000 확진자까지의 데이터를 크롤링하는 함수 

def get_seoul_covid19_past(page_num):
    
    start_no = (page_num - 3) * 100
    
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax_pre.php?draw={page_num}"
    url = f"{url}&order%5B0%5D%5Bdir%5D=desc&start={start_no}&length=100"
        
    response = requests.get(url)
    data_json = response.json()
    
    return data_json

In [ ]:
page_list = []
all_page = 1
end_page = 1201

for page_no in trange(all_page ,end_page):
    
    # 차단막는 코드, 랜덤으로 time.sleep 지정
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(3)
    time.sleep(a)
        
        
    df = get_seoul_covid19_past(page_no)
    #json 타입 데이터에서 'data' key와 매칭하는 value 데이터를 데이터프레임으로 변환
    
    df = pd.DataFrame(df["data"]) 
    page_list.append(df)

In [ ]:
# 크롤링한 결과를 한번에 결합시킬 수 있지만 데이터가 문제없이 크롤링 되었는지 살펴보기위해 따로 저장 
df = pd.concat(page_list)
df.tail(3)

In [ ]:
# 120001~ 현재 확진자까지의 데이터를 크롤링하는 함수

def get_seoul_covid19_today(page_num):
    
    start_no = (page_num - 3) * 100
    
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_num}"
    url = f"{url}&order%5B0%5D%5Bdir%5D=desc&start={start_num}&length=100"
        
    response = requests.get(url)
    data_json = response.json()
    
    return data_json

In [ ]:
page_list = []

#크롤링할 페이지 설정
all_page = 1
end_page = 514

for page_no in trange(all_page ,end_page):
    
    # 차단막는 코드, 랜덤으로 time.sleep 지정
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(3)
    time.sleep(a)
            
    df = get_seoul_covid19_today(page_num)
    
    #json 타입 데이터에서 'data' key와 매칭하는 value 데이터를 데이터프레임으로 변환
    df = pd.DataFrame(df["data"]) 
    page_list.append(df)

In [ ]:
df1 =pd.concat(page_list)
df1.tail(3)

In [ ]:
# past ,today DataFrame 결합 
df = pd.concat([df1,df], ignore_index =True)

In [ ]:
# csv파일로 저장 
df.to_csv("covid.csv")

### Preprocessing

In [ ]:
df = pd.read_csv("covid_csv")

In [ ]:
#정규표현식 사용을 위해 import re 
import re

In [ ]:
def extract_number(num_string):
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "") # 'corona19'를 제거한 후에 
        num = re.sub("[^0-9]", "", num_string)# 숫자가 아닌 문자 모두 제거 ,re.sub（정규 표현식, 대상 문자열 , 치환 문자）
        # [^0-9] not을 표현하며 0~9를 제외한 문자를 의미 
        
        num = int(num)
        return num
    else:
        return num_string

df["연번"] = df["연번"].map(extract_number)
df    

In [ ]:
def extract_hangeul(origin_text):
    subtract_text = re.sub("[^가-힣]", "", origin_text) # 한글이 아닌 문자 제거
    # [^가-힣] not을 표현하며 가~힣 를 제외한 문자를 의미 
    
    return subtract_text

df["퇴원현황"] = df["퇴원현황"].map(extract_hangeul)
df.loc[df["퇴원현황"].isin(['']), "퇴원현황"] = np.nan # 퇴원현황 값이 없으면 NaN으로 표기 
df

In [ ]:
df.to_csv("covid_data.csv")